In [230]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import copy
from numba import jit
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

In [200]:
np.set_printoptions(linewidth=np.inf)

In [201]:
class PWApprx:

    def __init__(self, x_values, r):
        self.x_values = x_values
        self.r = r
        self.y_values = np.multiply(x_values, r) * np.subtract(1, x_values)
        self.func_list = self.__create_func_list(x_values, self.y_values, r)  
        self.slope_list = self.__create_slope_list(x_values,self.y_values, r)      
    
    def compute(self, x):
        # clamp x to between zero and one
        x = max(min(x,1),0)
        idx = -1
        try:
            idx = min(int(x * len(self.func_list)), len(self.func_list)-1)
            return max(min(self.func_list[idx](x),1),0)
        except:
            print(f"idx={idx}  x={x}   r={self.r} func")
            return 0
        
    def slope(self, x):
        # clamp x to between zero and one
        x = max(min(x,1),0)
        idx = -1
        try:
            # multiply x by the length of the list conveted to an integer will give us what bin x should go to
            idx = min(int(x * len(self.slope_list)), len(self.slope_list)-1)
            return self.slope_list[idx]
        except:
            print(f"idx={idx}  x={x}   r={self.r} slope")
            return 0
    
    def get_x_values(self):
        return self.x_values
    
    def get_y_values(self):
        return self.y_values
    
    def get_func_list(self):
        return self.func_list
    
    def get_slope_list(self):
        return self.slope_list
    
    # private methods 
    
    def __create_func_list(self, x_values, y_values, r_value):
        func_list = []
        for i in range(len(x_values)):
            if i+1 < len(x_values):
                func_list.append(self.__func_factory(x_values[i],x_values[i+1],y_values[i],y_values[i+1]))  
        return func_list
    
    def __func_factory(self, x1,x2,y1,y2):
        return lambda x: ((y2-y1)/(x2-x1)) * (x - x1) + y1
    
    def __create_slope_list(self, x_values, y_values, r_value):
        slope_list = []
        for i in range(len(x_values)):
            if i+1 < len(x_values):
                slope_list.append((y_values[i+1]-y_values[i])/(x_values[i+1]-x_values[i]))
        return slope_list

In [275]:
def lyapunov_fractal(apprx_type = 'logistic', # determines which of the three kinds of approximations to use, linear, logistic, and invariant_linear
                     nmax = 120, # number of iterations used to compute the lyapunov exponents
                     n_warmups = 1200, # number of warmup iterations
                     a_lb = 2, a_ub = 4, # lower and upper bounds of the A axis
                     b_lb = 2 , b_ub = 4, # lower and upper bounds of the B axis
                     steps = 500, # the number of values between each upper and lower bound
                     x_0 = 0.5, # starting value of x
                     n_points = 5, # number of (internal) points used to construct a linear apprx
                     savepng = False, # save a png of the fractal image
                     savefile = False # save a binary file with the lyapunov data from the fractal
                    ):
    
    @jit
    def logisitc_derivative(x, r, epsilon = 0):
        return np.add(np.abs(np.multiply(rb, (np.subtract(1, (np.multiply(2, x.T))))).T), epsilon)
    
    @jit
    def linear_derivative(x, function):
        return np.abs(function.slope(x))
    
    @jit
    def calc_equal_portion(n, total_x_interval):
        p = (1/n) * total_x_interval
        return p

    # returns the weight (the proportion) of x values of a function mapped by y values of another fuction   
    @jit
    def calc_weight(x_lb, x_ub, y_lb, y_ub):
        overlap_lb = max(x_lb,y_lb)
        overlap_ub = min(x_ub,y_ub)

        if overlap_lb > overlap_ub:
            return 0
        else:
            w = (overlap_ub - overlap_lb) /(y_ub - y_lb)
        return w  
    
#     def invariant_lyapunov_AB(x_values, ra, rb):  

#         # create functions for linear approximation

#         func_a = PWApprx(x_values, ra)
#         func_b = PWApprx(x_values, rb)
#         y_values_a = func_a.get_y_values()
#         y_values_b = func_b.get_y_values()
#         matrix = []
#         p = []

#         total_x_interval = np.amax(x_values) - np.amin(x_values)


#         for i in range(len(x_values)):
#             if i != 0:

#                 w_i = []
#                 w_z = []

#                 for j in range(len(y_values_a)):
#                     if j != 0:
#                         # Top Left Square
#                         weight = 0
#                         if i == j:
#                             weight = weight - 1
#                         w_z.append(weight)

#                         # Top Right Square
#                         weight = calc_weight(x_values[i-1],x_values[i], min(y_values_b[j-1],y_values_b[j]),max(y_values_b[j-1],y_values_b[j]))
#                         w_i.append(weight)    

#                 matrix.append((w_z + w_i))
               
#                 # Proportions
#                 p.append(calc_equal_portion(len(x_values)-1, total_x_interval)/2)

#         for i in range(len(x_values)):
#             if i != 0:      

#                 w_i = [] 
#                 w_z = []

#                 for j in range(len(y_values_b)):
#                     if j != 0:  
#                         # Bottom Left Square
#                         weight = calc_weight(x_values[i-1],x_values[i], min(y_values_a[j-1],y_values_a[j]),max(y_values_a[j-1],y_values_a[j]))
#                         w_i.append(weight)
        
#                         # Bottom Right Square
#                         weight = 0
#                         if i == j:
#                             weight = weight - 1
#                         w_z.append(weight)  

#                 matrix.append((w_i + w_z))

#                 # Proportions
#                 p.append(calc_equal_portion(len(x_values)-1, total_x_interval)/2)


#         matrix = np.asarray(matrix)

#     #     for col in matrix.T:
#     #         print(sum(col))  

#     #     print(matrix) 

#         matrix[-1] = p

#         ans = np.zeros(len(matrix))
#         ans[-1] = 1

#     #     print(matrix) 
#     #     print(ans)
        
#         try:
#             solution = np.linalg.solve(matrix, ans)
#         except:
#             print(ra, rb)
#             print(matrix)
            
#         slopes_a = func_a.get_slope_list()
#         slopes_b = func_b.get_slope_list()

#         slopes = slopes_a + slopes_b

#     #     print(slopes)

#         lyap = np.sum(p * solution * np.log(np.abs(slopes)))

#         return lyap
    
    start = time.time()

    ra = np.linspace(a_lb, a_ub, steps)
    rb = np.linspace(b_lb, b_ub, steps)
    
    x_values = np.round(np.linspace(0, 1, n_points+2), 3) # values of x on the curve that will be used to create the approximation
    
    lyapunov_grid = []
    
#     for b_i, b in enumerate(rb):
#         if b_i % 10 == 0:
#             print(b)
#         lyapunov_row = []
#         for a_i, a in enumerate(ra):
#             lyapunov_row.append(invariant_lyapunov_AB(x_values, a, b))
#         lyapunov_grid.append(lyapunov_row)
    
    total_x_interval = np.amax(x_values) - np.amin(x_values)
    
    
    # create functions for linear approximation
    func_list_a = [0 for a in range(len(ra))]
    
    for a_i, a in enumerate(ra):
        # create the piecewise functions for that ra value
        func_A = PWApprx(x_values, a)
        func_list_a[a_i] = func_A

    if not a_lb == b_lb and a_ub == b_ub:
        func_list_b = [0 for b in range(len(rb))]

        for b_i, b in enumerate(rb):
            func_B = PWApprx(x_values, b)
            func_list_b[b_i] = func_B        
    else:
        func_list_b = func_list_a
    
    n_equations = len(x_values)-1
    
    zeros = np.zeros((n_equations,n_equations))
    for z_i, z in enumerate(zeros):
        z[z_i] = -1
    
    p = [(calc_equal_portion(n_equations, total_x_interval)/2) for i in range(n_equations * 2)]
    ans = np.zeros(len(p))
    ans[-1] = 1
    bottoms = []
    
    @jit
    def lyapunov_from_matrix(matrix, slopes):
        solution = np.linalg.solve(matrix, ans)
    
#        solution = scipy.linalg.solve(matrix, ans, check_finite = False, overwrite_a = True)
    
#         matrix = csc_matrix(matrix)
#         solution = spsolve(matrix, ans)
    
        return np.sum(p * solution * np.log(np.abs(slopes)))
    
    # counts the number of times a singular matrix is encountered
    singularities = 0
    # counts the number of times a lyap exponent is far below the normal, possibly caused by an approach to negative infinity in the log
    negative_outliers = 0
    
    for b_i, b in enumerate(rb):
        
        y_values_b = func_list_b[b_i].get_y_values()
        top_right_square = []
        
        for i in range(len(x_values)):
            if i != 0:      
                w_i = []
                for j in range(len(x_values)):
                    if j != 0:  
                        # Top Right Square
                        weight = calc_weight(x_values[i-1],x_values[i], min(y_values_b[j-1],y_values_b[j]),max(y_values_b[j-1],y_values_b[j]))
                        w_i.append(weight)
                top_right_square.append(w_i)
        
        if b_i % (max(int(steps/20), 1)) == 0:
            print("b = {:1.3}, {:2.2%} done".format(b, (b_i/steps)))
        lyapunov_row = []
        
        # needed to compute lyapunov exponent
        slopes_b = func_list_b[b_i].get_slope_list()
        
        top = np.concatenate((zeros, top_right_square), axis = 1)
        
        for a_i, a in enumerate(ra):
            
            if b_i == 0:
                y_values_a = func_list_b[a_i].get_y_values()
                matrix = []

                for i in range(len(x_values)):
                    if i != 0:      
                        w_i = []
                        for j in range(len(x_values)):
                            if j != 0:  
                                # Bottom Left Square
                                weight = calc_weight(x_values[i-1],x_values[i], min(y_values_a[j-1],y_values_a[j]),max(y_values_a[j-1],y_values_a[j]))
                                w_i.append(weight) 
                        matrix.append(w_i)
                
                bottom = np.concatenate((matrix, zeros), axis = 1)
                bottoms.append(bottom)
            
            matrix = np.concatenate((top, bottoms[a_i]))
            matrix[-1] = p
            
            slopes_a = func_list_a[a_i].get_slope_list()
            slopes = slopes_a + slopes_b
            
            try:
                lyapunov_ex = lyapunov_from_matrix(matrix, slopes)
                
                if lyapunov_ex < -4:
#                     print(a, b)
#                     print(matrix)
                    negative_outliers = negative_outliers + 1
                    lyapunov_ex = -4
            except:
#                 print(a, b)
#                 print(matrix)
                singularities = singularities + 1
                lyapunov_ex = 0    
        
#             lyapunov_ex = lyapunov_from_matrix(matrix, slopes)
            
            lyapunov_row.append(lyapunov_ex)
            
        lyapunov_grid.append(lyapunov_row)
    
    
    print("Singularities: " + str(singularities))
    print("Negative Outliers: " + str(negative_outliers))
    
    if savefile:
        f = open("invariat_fractal_apprx n_points= " + str(n_points) + " steps=" + str(steps) + ".dat", "wb")
        lyapunov_grid = np.ascontiguousarray(lyapunov_grid)
        f.write(lyapunov_grid)
        f.close()
    
    # plot information
    
    fig, ax = plt.subplots(figsize=(10, 9))
    
    fig.patch.set_facecolor('white')
    
#     lya_cmap = copy.copy(mpl.cm.get_cmap('viridis'))
    lya_cmap = copy.copy(mpl.cm.get_cmap('jet'))
    lya_cmap.set_over('black')
#     lya_cmap.set_under('white')
#     lya_cmap.set_bad('black')
    
    plt.imshow(lyapunov_grid , cmap = lya_cmap, origin = 'lower', vmax = 0, vmin = -4)
    
    plt.colorbar()
    plt.suptitle("Lyapunov Logistic Fractal - Linear Apprx")
    plt.title("steps = " + str(steps) + " n_points = " + str(n_points))  
    
    xticks = np.arange(a_lb, a_ub, .2)
    yticks = np.arange(b_lb, b_ub, .2)
    
    plt.xlabel("A")  
    plt.ylabel("B") 
    
    plt.xticks([])
    plt.yticks([])
    
    if(savepng):
        plt.savefig("invariat_fractal_apprx n_points = "+ str(n_points) +".png")
    
    plt.show()
    
    end = time.time()

    print("This took", end - start, "seconds to execute")
    

In [ ]:
%prun lyapunov_fractal(n_points = 251, steps = 500, savepng = True, savefile = True)

b = 2.0, 0.00% done


<ipython-input-275-9ffaad93622c>:179: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "lyapunov_from_matrix" failed type inference due to: No implementation of function Function(<built-in function mul>) found for signature:
 
 >>> mul(reflected list(float64)<iv=None>, array(float64, 1d, C))
 
There are 12 candidate implementations:
      - Of which 10 did not match due to:
      Overload of function 'mul': File: <numerous>: Line N/A.
        With argument(s): '(reflected list(float64)<iv=None>, array(float64, 1d, C))':
       No match.
      - Of which 2 did not match due to:
      Operator Overload in function 'mul': File: unknown: Line unknown.
        With argument(s): '(reflected list(float64)<iv=None>, array(float64, 1d, C))':
       No match for registered cases:
        * (int64, int64) -> int64
        * (int64, uint64) -> int64
        * (uint64, int64) -> int64
        * (uint64, uint64) -> uint64
        * (float32, float32

b = 2.1, 5.00% done
